# Instrumental variables

## Baby example on mock dataset

### Constructing the dataset

Create four random series of length $N=1000$

- $x$: education
- $y$: salary
- $z$: ambition
- $q$: early smoking 

such that:

1. $x$ and $z$ cause $y$
2. $z$ causes $x$
3. $q$ is correlated with $x$, not with $z$

(all relations are linear, add random shocks where needed)

We want to study the effect of $x$ on $y$.

__Run the following code to create a mock dataset.__

In [25]:
import numpy
import pandas
N = 100000

In [26]:
ϵ_z = numpy.random.randn(N)*0.1
ϵ_x = numpy.random.randn(N)*0.1
ϵ_q = numpy.random.randn(N)*0.1
ϵ_y = numpy.random.randn(N)*0.1

In [27]:
z = 0.1 + ϵ_z
x = 0.1 + 0.5*z + ϵ_x
q = 0.5 + 0.1234*ϵ_x + ϵ_q
y  = 1.0 + 0.9*x + 0.4*z + ϵ_y

In [28]:
df = pandas.DataFrame({
    "x": x,
    "y": y,
    "z": z,
    "q": q
})

__Describe the data. Compute the correlations between series.__

Here are the results from the database:

In [29]:
df.describe()

,x,y,z,q
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.149838,1.174453,0.099544,0.499990
std,0.111716,0.158766,0.099791,0.100681
min,-0.346680,0.487024,-0.307697,0.100623
25%,0.074457,1.067474,0.032258,0.431809
50%,0.149622,1.174309,0.099331,0.500190
75%,0.225062,1.280778,0.166741,0.567860
max,0.711602,1.836777,0.528661,0.924587


In [30]:
df.corr()

,x,y,z,q
x,1.000000,0.745872,0.446562,0.106993
y,0.745872,1.000000,0.534009,0.067097
z,0.446562,0.534009,1.000000,-0.003369
q,0.106993,0.067097,-0.003369,1.000000


We observe:
- cor(q, x) non zero: the instrument is relevant
    - close to zero: might be a weak instrument (we would need to check significance)
- cor(q, z) = 0 : the instrument is really exogenous

### OLS Regression

Run a regression to estimate the effect of $x$ on $y$. Compare the result using `statsmodels` and `linearmodels`.

What is the problem with this regression? How can it be detected?



In [31]:
from linearmodels.iv import IV2SLS

formula = "y ~ x"
model = IV2SLS.from_formula(formula, df)
result = model.fit()
result

OLS Estimation Summary                            
==============================================================================
Dep. Variable:                      y   R-squared:                      0.5563
Estimator:                        OLS   Adj. R-squared:                 0.5563
No. Observations:              100000   F-statistic:                 1.234e+05
Date:                Mon, Mar 27 2023   P-value (F-stat)                0.0000
Time:                        21:54:41   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.0156     0.0006     1800.8     0.0000      1.0145      1.0167
x              1.0600     0.0030     351.22     0.0000      1.0541      1.0659
==============================================================================
OLSResults, id: 0x7f5789b06260

Regression is globally significant (p-value for Fisher test < 0.00001).
The coefficient $\beta=1.0999$ in front of $x$ is also very significant at a 0.001% level but does not match the model (`y  = 1.0 + 0.9*x + 0.4*z + ϵ_y`)

### Regress again $y$ on $x$, this time controling for missing variable $z$.



In [32]:
from linearmodels.iv import IV2SLS

formula = "y ~ x + z"
model = IV2SLS.from_formula(formula, df)
result = model.fit()
result

OLS Estimation Summary                            
==============================================================================
Dep. Variable:                      y   R-squared:                      0.6068
Estimator:                        OLS   Adj. R-squared:                 0.6067
No. Observations:              100000   F-statistic:                 1.518e+05
Date:                Mon, Mar 27 2023   P-value (F-stat)                0.0000
Time:                        21:54:44   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.9997     0.0005     1818.9     0.0000      0.9987      1.0008
x              0.9007     0.0032     283.90     0.0000      0.8945      0.9069
z              0.3993     0.0035     113.00     0.0000      0.3924      0.4062
==============================================================================
OLSResults, id: 0x7f5788c9ea10

Now we see that the coefficient in front of `x` is the correct one (that is 0.9).

### Instrumental variable

Make a causality graph, summarizing what you know from the equations.




Use $q$ to instrument the effect of x on y. Comment.

In [36]:
from linearmodels.iv import IV2SLS

formula = "y ~ 1 + [x ~ q]"
model = IV2SLS.from_formula(formula, df)
result = model.fit()
result

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                      y   R-squared:                      0.5422
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5422
No. Observations:              100000   F-statistic:                    979.40
Date:                Mon, Mar 27 2023   P-value (F-stat)                0.0000
Time:                        21:56:10   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.0409     0.0043     243.27     0.0000      1.0325      1.0493
x              0.8912     0.0285     31.295     0.0000      0.8354      0.9470
==============================================================================

Endogenous: x
Instruments: q
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x7f5784cd9d80

We observe that the result is, again, the correct one. This is especially impressive since *we didn't have access to the confounding factor `z`* and couldn't add it to the regression. Instead, we had another source of randomness `q` that we used to instrument the regression.

## Return on Education

We follow the excellent R [tutorial](https://www.econometrics-with-r.org/12-6-exercises-10.html) from the (excellent) *Econometrics with R* book.

The goal is to measure the effect of schooling on earnings, while correcting the endogeneity bias by using distance to college as an instrument.

__Download the college distance using `get_dataset` function and make a nice dataframe. Describe the dataset. Plot a histogram of distance (you can use `matplotlib`'s `hist` function or `seaborn`).__

https://vincentarelbundock.github.io/Rdatasets/datasets.html

In [37]:
import statsmodels.api as sm
ds = sm.datasets.get_rdataset("CollegeDistance", "AER")
df = ds.data

In [38]:
df.head()

,gender,ethnicity,score,fcollege,mcollege,home,urban,unemp,wage,distance,tuition,education,income,region
1,male,other,39.150002,yes,no,yes,yes,6.2,8.09,0.2,0.88915,12,high,other
2,female,other,48.869999,no,no,yes,yes,6.2,8.09,0.2,0.88915,12,low,other
3,male,other,48.740002,no,no,yes,yes,6.2,8.09,0.2,0.88915,12,low,other
4,male,afam,40.400002,no,no,yes,yes,6.2,8.09,0.2,0.88915,12,low,other
5,female,other,40.480000,no,no,no,yes,5.6,8.09,0.4,0.88915,13,low,other


In [35]:
df.describe()

,score,unemp,wage,distance,tuition,education
count,4739.000000,4739.000000,4739.000000,4739.000000,4739.000000,4739.000000
mean,50.889029,7.597215,9.500506,1.802870,0.814608,13.807765
std,8.701910,2.763581,1.343067,2.297128,0.339504,1.789107
min,28.950001,1.400000,6.590000,0.000000,0.257510,12.000000
25%,43.924999,5.900000,8.850000,0.400000,0.484990,12.000000
50%,51.189999,7.100000,9.680000,1.000000,0.824480,13.000000
75%,57.769999,8.900000,10.150000,2.500000,1.127020,16.000000
max,72.809998,24.900000,12.960000,20.000000,1.404160,18.000000


Create a binary variable `incomeb` which equals 1 if `income` is `high`, 0 otherwise.

In [40]:
df['income'].unique()

array(['high', 'low'], dtype=object)

In [41]:
# option 1
df['incomeb'] = df['income'].map({'high' : 1, 'low': 0})

In [42]:
# option 2
df['incomeb'] = (df['income'] == 'high')*1

__Run the naive regression $\text{incomeb}=\beta_0 + \beta_1 \text{education} + u$ using linearmodels. Comment.__



In [43]:
from linearmodels.iv import IV2SLS

formula = "incomeb ~ education"
model = IV2SLS.from_formula(formula, df)
result = model.fit()
result

OLS Estimation Summary                            
==============================================================================
Dep. Variable:                incomeb   R-squared:                      0.0480
Estimator:                        OLS   Adj. R-squared:                 0.0478
No. Observations:                4739   F-statistic:                    227.43
Date:                Mon, Mar 27 2023   P-value (F-stat)                0.0000
Time:                        21:58:28   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -0.4780     0.0499    -9.5702     0.0000     -0.5759     -0.3801
education      0.0555     0.0037     15.081     0.0000      0.0483      0.0627
==============================================================================
OLSResults, id: 0x7f57819dd9c0

We find that education explains higher income with a significant, but low coefficient 0.05.


__Augment the regression with `unemp`, `hispanic`, `af-am`, `female` and `urban`. (Hint: you can use the `C()` function to deal with categorical data.__

In [45]:
df['gender']

1         male
2       female
3         male
4         male
5       female
         ...  
9391      male
9401      male
9411      male
9421      male
9431      male
Name: gender, Length: 4739, dtype: object

In [46]:
df.columns

Index(['gender', 'ethnicity', 'score', 'fcollege', 'mcollege', 'home', 'urban',
       'unemp', 'wage', 'distance', 'tuition', 'education', 'income', 'region',
       'incomeb'],
      dtype='object')

In [47]:
# needed only if you use the function Treatment in the formulas
from patsy import Treatment

from linearmodels.iv import IV2SLS

formula = "incomeb ~ education + unemp + C(gender) + C(ethnicity)"
model = IV2SLS.from_formula(formula, df)
result = model.fit()
result

OLS Estimation Summary                            
==============================================================================
Dep. Variable:                incomeb   R-squared:                      0.0811
Estimator:                        OLS   Adj. R-squared:                 0.0802
No. Observations:                4739   F-statistic:                    443.38
Date:                Mon, Mar 27 2023   P-value (F-stat)                0.0000
Time:                        22:00:32   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                                    Parameter Estimates                                     
============================================================================================
                          Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------------
Intercept                   -0.4361     0.0533    -8.1797     0.0000     -0.5406     -0.3316
C(ethnicity)[T.hispanic]    -0.0249     0.0185    -1.3425     0.1794     -0.0612      0.0114
C(ethnicity)[T.other]        0.1347     0.0163     8.2871     0.0000      0.1029      0.1666
C(gender)[T.male]            0.0484     0.0128     3.7866     0.0002      0.0234      0.0735
education                    0.0511     0.0037     13.982     0.0000      0.0439      0.0582
unemp                       -0.0111     0.0022    -4.9609     0.0000     -0.0155     -0.0067
============================================================================================
OLSResults, id: 0x7f5781306440

In [48]:
df['ethnicity'].unique()

array(['other', 'afam', 'hispanic'], dtype=object)

__Comment the results and explain the endogeneity problem__

Adding additional regressors has increased the fit (adj. R^2 from 0.04 to 0.08) without changing the coefficient on the education level. This would imply that regression is robust.

However, we might have an endogeneity issue with some potential other factors explaining both income level and salary (cf many discussions in the course).

__Explain why distance to college might be used to instrument the effect of schooling.__

Assuming that the decision to live in 
a given county does not depend on the presence of a college nearby, the distance to college should be exogenous.

The distance to college is probably correlated with the decision to go so the instrument should have some power (opposite of weak)


__Run an IV regression, where `distance` is used to instrument schooling.__

look at: 
    https://bashtage.github.io/linearmodels/
   (two-stage least squares)

In [58]:
# needed only if you use the function Treatment in the formulas
from patsy import Treatment

from linearmodels.iv import IV2SLS
 
formula = "incomeb ~ [education ~ distance] + unemp + C(gender) + C(ethnicity)"
model = IV2SLS.from_formula(formula, df)
result = model.fit()
result

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                incomeb   R-squared:                     -0.1339
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1351
No. Observations:                4739   F-statistic:                    1748.5
Date:                Wed, Mar 01 2023   P-value (F-stat)                0.0000
Time:                        11:06:52   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                                    Parameter Estimates                                     
============================================================================================
                          Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------------
C(ethnicity)[T.afam]        -2.0318     0.4856    -4.1842     0.0000     -2.9836     -1.0801
C(ethnicity)[T.hispanic]    -2.0813     0.4929    -4.2226     0.0000     -3.0474     -1.1152
C(ethnicity)[T.other]       -1.9566     0.5039    -3.8830     0.0001     -2.9441     -0.9690
C(gender)[T.male]            0.0457     0.0142     3.2195     0.0013      0.0179      0.0735
unemp                       -0.0100     0.0025    -3.9704     0.0001     -0.0149     -0.0051
education                    0.1692     0.0358     4.7261     0.0000      0.0990      0.2393
============================================================================================

Endogenous: education
Instruments: distance
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x7f0acc0f8a30

__Comment the results. Compare with the R tutorials.__

The estimate we get for the return on education is three times higher and highly significant.